<a href="https://colab.research.google.com/github/rani-ops/medical-chatbot/blob/main/chatbot_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pypdf
!pip install langchain
!pip install torch
!pip install accelerate
!pip install bitsandbytes
!pip install transformers
!pip install sentence_transformers
!pip install faiss_cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=9d5b58faf61d8217e62a933b32c922f45

In [2]:
!pip install chainlit
!pip install typing_extensions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:

from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
import chainlit as cl

DB_FAISS_PATH = "vectorstores/db_faiss"

custom_prompt_template = """use the following pieces of information to answer the user's question.
if you don't know the answer, please just say that you don't know the answer, don't try to make up an answer.

Context: {}
Question: {question}

only returns the helpful answer below and nothing else.
Helpful answer:
"""

def set_custom_prompt():
  """
  prompt template for QA retrieval for ecah vector stores
  """
  prompt = PromptTemplate(template=custom_prompt_template, input_variable=['context','question'])

  return prompt

def load_llm():
  llm = CTransformers(
      model = "llama-2-7b-chat.ggmlv3.q8_0.bin",
      model_type = "llama",
      max_new_token = 512,
      temperature = 0.5
  )
  return llm

def retrieval_qa_chain(llm, prompt, db):
  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = db.as_retriever(search_kwargs={'k':2}),
      return_source_documents = True,
      chain_type_kwargs = {'prompt': prompt}
  )
  return qa_chain

def qa_bot():
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                     model_kwargs={'device':'cpu'})
  db = FAISS.load_local(DB_FAISS_PATH,embeddings)
  llm = load_llm()
  qa_prompt = set_custom_prompt()
  qa = retrieval_qa_chain(llm,qa_prompt,db)

  return qa

def final_result(query):
  qa_result = qa_bot()
  response = qa_result({'query': query})
  return response

@cl.on_chat_start
async def start():
  chain = qa_bot()
  msg = cl.Message(content="Starting the bot.....")
  await msg.send()
  msg.content = "Hi, Welcome to the Medical Bot. What is your query?"
  await msg.update()
  cl.user_session.set("chain", chain)

@cl.on_message
async def main(message):
  chain = cl.user_session.set("chain")
  cb = cl.AsyncLangChainCallbackHandler(
      stream_final_answer = True, answer_prefix_tokens = ["FINAL", "ANSWER"]
  )
  cb.answer_reached=True
  res = await chain.acall(message, callbacks=[cb])
  answer = res["result"]
  sources = res["source_documents"]

  if sources:
    answer += f"\nSources:" + str(sources)
  else:
    answer += f"\nNo Sources Found"

  await cl.Message(content=answer).send()



